# Bootstrapping with Coulomb Potential

In this code, we demonstrate radial Coulomb potential with $\hbar = 1$ and $m_e = 1$. We begin with

\begin{align*}
    H &= \frac{P_r^2}{2m_e} + \frac{l(l+1)}{2m_er^2}\hbar^2 + V(r)\\
    V(r) &= -\frac{e^2}{4\pi\epsilon_0r} = -\frac{k}{r}\\
    [r,P_r] &= i
\end{align*}

We see that the Hamiltonian is similar to 1-D Harmonic potential, with additional angular term $\frac{l(l+1)}{2r^2}$. So we just need to consider the additional commutation relation of the additional angular term. (Denote $P_r=P$ in the following)

\begin{align*}
    [H, r^{s}P] &= 0 = 
    -\frac{1}{2}s(s-1)\langle r^{s-2}P\rangle 
    -is\langle r^{s-1}P^2\rangle
    +i\langle r^{s}V^\prime(r)\rangle
    -il(l+1)\langle r^{s-3}\rangle\\
    
    [H, r^{s-1}] &= 0 =
    -\frac{1}{2}(s-1)(s-2)\langle r^{s-3}\rangle
    -i(s-1)\langle r^{s-2}P\rangle
\end{align*}

With the same trick and the same algebraic calcukation, we get the final recursion relation
\begin{equation*}
    0 = 2sE\langle r^{s-1}\rangle + \frac{1}{4}s(s-1)(s-2)\langle r^{s-3}\rangle 
    - \langle r^s V^\prime (r)\rangle - 2s\langle r^{s-1}V(r)\rangle -(s-1)l(l+1)\langle r^{s-3}\rangle
\end{equation*}

In [1]:
import os, time
import numpy as np
import sympy as sp
from bootstrap_sympy import sympy_solve_intervals, plot_energy_interval

import matplotlib.pyplot as plt
import seaborn as sns

## Recursion relation for 3-d Coulomb potential

Plug in $V(r)=-\frac{k}{r}$ into the recursion relation, we get

\begin{equation*}
    8(s+1)E\langle r^s\rangle = -4(2s+1)k\langle r^{s-1}\rangle
    + 4sl(l+1)\langle r^{s-2}\rangle - s(s+1)(s-1)\langle r^{s-2}\rangle
\end{equation*}

Take $s=0$ we get $\langle r^{-1}\rangle=-\frac{2E}{k}$ . If $s>0$

\begin{equation*}
    -E\langle r^s\rangle = \frac{k(2s+1)}{2(s+1)}\langle r^{s-1}\rangle
    + [\frac{s(s-1)}{8} - \frac{sl(l+1)}{2(s+1)}]\langle r^{s-2}\rangle
\end{equation*}

Since the energy eigenvalue is expected to be negative, and in order to transform it into a polynomial recursion in $E$, we set $E^\prime = -\frac{1}{E}$, and just denote $E^\prime$ as $E$ in the code below. With $\langle r^0\rangle=1$ by normalization, take $s=1$, we get $\langle r\rangle=\frac{3}{4}kE-\frac{l(l+1)}{2k}$. Finally, rewrite the recursion as

\begin{equation*}
    \langle r^s\rangle = \frac{k(2s+1)}{2(s+1)}E\langle r^{s-1}\rangle
    + [\frac{s(s-1)}{8} - \frac{sl(l+1)}{2(s+1)}]E\langle r^{s-2}\rangle
\end{equation*}

In [2]:
class ColumbPotentialMatrix:
    def __init__(self, N, angular_momentum=True):
        self.N = N # maximum size of submatrix
        self.k = sp.symbols('k') # constant of harmonic potential
        self.E = sp.symbols('E') # eigen-energy to be solved (Note the true energy is 1/E)
        self.l = sp.symbols('l') # angular momentum quantum number
        self.angular_momentum = angular_momentum
        
    def evaluate(self):
        self.rs_list = [] # list of r^s, the expectation value of position operator r to the s power
        for i in range(2*(self.N-1)+1):
            if i >= 2:
                self.rs_list.append(self.rs_recursion(s=i, rs_1=self.rs_list[i-1], rs_2=self.rs_list[i-2]))
            else:
                self.rs_list.append(self.rs_recursion(s=i))

    def rs_recursion(self, s, rs_1=None, rs_2=None):
        # find the <r^s> with recursion relation
        if s == 0:
            return 1
        elif s == 1:
            result = sp.Rational(3,4) * self.k * self.E
            if self.angular_momentum:
                result += -sp.Rational(1,2) * (self.l*(self.l+1)) / self.k
            return result
        else:
            if rs_1 == None or rs_2 == None:
                rs_1 = self.rs_recursion(s-1)
                rs_2 = self.rs_recursion(s-2)
            # 1/E -> -E
            result  = self.k * sp.Rational(2*s+1,2*s+2) * self.E * rs_1
            result += sp.Rational(s*(s-1),8) * self.E * rs_2
            if self.angular_momentum:
                result += -sp.Rational(s*self.l*(self.l+1),2*s+2) * self.E * rs_2
            return result

    def submatrix(self, L):
        # L*L matrix's [i,j] element = <r^(i+j)>
        return sp.Matrix([[self.rs_list[i+j] for j in range(L)] for i in range(L)])

In [3]:
cp_config = {
    'x_inf' : 0,
    'x_sup' : 10,
    'round' : 15,
    'plot_step' : 2,
    'threshold' : 1e-2,
    'initial_interval' : sp.Interval(-sp.oo, sp.oo),
}

cp_matrix = ColumbPotentialMatrix(N=cp_config['round'], angular_momentum=False)
cp_matrix.k = 1
cp_matrix.l = 0
cp_matrix.evaluate()

energy_intervals, confirmed_intervals = sympy_solve_intervals(cp_matrix, cp_config, det_indep=False)

Now calculating determinant size 1x1
Time cost = 0.00
Round interval = Interval(-oo, oo)

Now calculating determinant size 2x2
Time cost = 0.01
Round interval = Union(Interval(-oo, -4.00000000000000), Interval(0, oo))

Now calculating determinant size 3x3
Time cost = 0.05
Round interval = Union(Interval(-oo, -64.1401474002245), Interval(0, oo))

Now calculating determinant size 4x4
Time cost = 0.16
Round interval = Union(Interval(-oo, -394.293936497373), Interval(0, oo))

Now calculating determinant size 5x5
Time cost = 0.50
Round interval = Union(Interval(-oo, -1560.58407263828), Interval(0, oo))

Now calculating determinant size 6x6
Time cost = 1.18
Round interval = Union(Interval(-oo, -4748.14977266714), Interval(0, oo))

Now calculating determinant size 7x7
Time cost = 2.89
Round interval = Union(Interval(-oo, -12104.1418769785), Interval(0, oo))

Now calculating determinant size 8x8
Time cost = 6.92
Round interval = Union(Interval(-oo, -27156.9138967942), Interval(0, oo))

Now cal